# Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) data structure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on a trie;
3. Measure suggestion speed;

![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [1]:
# !pip install pygtrie


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Build a trie upon a dataset

### 1.1. [5] Read the dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset. Pass the assert.

In [2]:
import pandas as pd

aol_data = None

# Open data file and read the content as pandas dataframe
file_path = '../datasets/aol/user-ct-test-collection-01.txt'
aol_data = pd.read_csv(file_path, sep='\t')

assert aol_data.shape[0] == 3558411, "Dataset size does not match"

In [3]:
aol_data.head()

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,NaN,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,NaN,NaN
2,142,staple.com,2006-03-17 21:19:29,NaN,NaN
3,142,staple.com,2006-03-17 21:19:45,NaN,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,NaN,NaN


### 1.2. [10] Build a Trie

We want a suggest function to be **non-sensitive to stop words** because we don't want to upset the users if they confuses/omits prepositions. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build a Trie based on the dataset, **storing query statistics such as query _frequency_, urls and ranks in the nodes**. Some queries may have no associated urls, others may have multiple ranked urls. Think of the way to store this information.

Pass the asserts.

In [4]:
class Node:
    frequency = 0
    urls = []
    ranks = []

In [13]:
import pygtrie
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

aol_trie = pygtrie.CharTrie()
stop_words = set(stopwords.words('english'))

#TODO: build a trie based on the dataset
index = 0

for index, row in aol_data.iterrows():
    tokenized = [w for w in word_tokenize(str(row['Query']).lower()) if not w in stop_words]
    # print(' '.join(tokenized))
    key = ' '.join(tokenized)
    node = Node()
    if key in aol_trie:
        node = aol_trie[key]

    node.frequency += 1
    # print(index, row['Query'], row['ClickURL'])
    # if pd.isnull(row['ClickURL']):
    #     print('None')

    if not pd.isnull(row['ClickURL']):
        # print('YES')
        node.urls.append(row['ClickURL'])
        node.ranks.append(row['ItemRank'])

    aol_trie[key] = node


# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a property of list type. But you can do something different. 
    bag += val.urls
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ <__main__.Node object at 0x00000224462B3490>
sample questions immigration interview ~ <__main__.Node object at 0x0000022445DD71F0>
sample questions interview ~ <__main__.Node object at 0x0000022445DD82B0>
sample questions family interview ~ <__main__.Node object at 0x0000022445DD8550>
sample questions sociology race ethnicity ~ <__main__.Node object at 0x0000022445DFF4F0>
sample questions biology ~ <__main__.Node object at 0x0000022445E79420>
sample questions us citizenship test ~ <__main__.Node object at 0x00000224442EA320>
sample questionarie teaching evaluation ~ <__main__.Node object at 0x00000224896515A0>
sample questionnaire teaching evaluation ~ <__main__.Node object at 0x000002248964EB90>
sample questionnaire clinical research coordinators certification ~ <__main__.Node object at 0x000002249F705C30>


## 2. [15] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Pass the asserts.

Question for analysis: What is the empirical threshold for minimal prefix for suggest?

In [16]:
stops = set('a on at of to is from for and with using the in &'.split())

In [ ]:
def complete_user_query(query: str, trie, top_k=5) -> list[str]:
    #TODO: suggest top_k options for a user query
    # sort results by frequency (!), 
    # suggest the QUERIES for first k ranked urls if available
    pass

        
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie)
print(res)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

assert "boys and girls club of conyers georgia" \
            in complete_user_query("boys girls club conyers", aol_trie, 10), "Should be here"

## 3. Measure suggest speed ##

### 3.1. [10] Full Trie test

Check how fast your search is working. Consider changing your code if it takes too long on average.

Sucess criterion:
- there is an average and a standard deviation for **multiple runs** of the given bucket.
- there is an average and a standard deviation for **multiple runs** of naive search in the unindexed dataset.

In [ ]:
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure average execution time and standard deviation (in milliseconds) per query and print it out
# Repeat this for index and for no index.

## 4. [10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

You can use use [pyspellchecker](https://pypi.org/project/pyspellchecker/) `candidates()` call, or use any other spellchecker implementation.

In [ ]:
def complete_user_query_with_spellchecker(query, trie, top_k=5) -> list[str]:
    #TODO: suggest top_k options for a user query
    # sort results by frequency (!!), 
    # suggest the QUERIES for first k ranked urls if available
    pass

In [ ]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, trie, 5) == \
            complete_user_query_with_spellchecker(q, trie, 5), "Assert {} and {} give different results".format(q, qc)